# Levenshtein distance and spelling corrections

In [1]:
from datasets import load_dataset
import time

c:\Users\katar\miniconda3\envs\morfeusz\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")['corpus']

In [3]:
corpus = corpus.map(lambda x: {'text': x['text'].lower()})

In [4]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
nlp = Polish()


### Compute frequency list for each of the processed files.

In [5]:
docs_frequency_list = []
for doc in corpus:
    doc_dict = {}
    tokens = nlp(doc['text'])
    for token in tokens:
        if token.text in doc_dict:
            doc_dict[token.text] += 1
        else:
            doc_dict[token.text] = 1
    docs_frequency_list.append(doc_dict)
docs_frequency_list[0]

{'nie': 5,
 'mówię': 1,
 ',': 5,
 'że': 3,
 'podoba': 1,
 'mi': 1,
 'się': 2,
 'też': 1,
 'pomysł': 1,
 'szkolenia': 1,
 'w': 4,
 'miejscu': 1,
 'pracy': 1,
 'ale': 1,
 'możesz': 1,
 'oczekiwać': 1,
 'firma': 1,
 'to': 2,
 'zrobi': 1,
 '.': 3,
 'szkolenie': 1,
 'pracowników': 1,
 'ich': 3,
 'praca': 1,
 '–': 1,
 'oni': 1,
 'tworzą': 1,
 'oprogramowanie': 1,
 'być': 1,
 'może': 1,
 'systemy': 1,
 'edukacyjne': 1,
 'stanach': 1,
 'zjednoczonych': 1,
 '(': 1,
 'lub': 1,
 'studenci': 1,
 ')': 1,
 'powinny': 1,
 'trochę': 1,
 'martwić': 1,
 'o': 1,
 'zdobycie': 1,
 'umiejętności': 1,
 'rynkowych': 1,
 'zamian': 1,
 'za': 1,
 'ogromne': 1,
 'inwestycje': 1,
 'edukację': 1,
 'zamiast': 1,
 'wychodzić': 1,
 'z': 1,
 'tysiącami': 1,
 'zadłużonych': 1,
 'studentów': 1,
 'i': 1,
 'narzekać': 1,
 'są': 1,
 'do': 1,
 'niczego': 1,
 'wykwalifikowani': 1}

### Aggregate the result to obtain one global frequency list. 

In [6]:
global_frequency_list = {}
for doc_dict in docs_frequency_list:
    for key, val in doc_dict.items():
        if key in global_frequency_list:
            global_frequency_list[key] += val
        else:
            global_frequency_list[key] = val
global_frequency_list_sorted = dict(sorted(global_frequency_list.items(), key=lambda item:item[1], reverse=True))
global_frequency_list_sorted

{',': 611388,
 '.': 399567,
 'w': 175120,
 'nie': 131355,
 'i': 126807,
 'na': 118984,
 'to': 116324,
 'z': 96918,
 'jest': 93226,
 'że': 90021,
 'się': 85835,
 'do': 66595,
 ')': 56954,
 'jeśli': 51706,
 '(': 48032,
 '„': 41330,
 'ale': 41228,
 'a': 41224,
 'o': 38193,
 '”': 37273,
 '-': 36409,
 'są': 35417,
 'jak': 33649,
 'lub': 33215,
 'za': 32311,
 '*': 31495,
 'aby': 31031,
 'od': 29828,
 'co': 29697,
 '"': 28966,
 '/': 28181,
 '?': 27034,
 'może': 26331,
 'po': 25687,
 'tak': 25583,
 'dla': 25446,
 'które': 23983,
 'możesz': 23208,
 ':': 22677,
 'tego': 21928,
 'tym': 21604,
 '%': 20830,
 'ma': 20278,
 'czy': 20269,
 'być': 19261,
 'ponieważ': 18944,
 'przez': 17831,
 'usd': 17509,
 'tylko': 17255,
 'więc': 17076,
 'niż': 16827,
 'ich': 14567,
 'pieniądze': 13932,
 'więcej': 13828,
 'gdy': 13526,
 'jako': 12974,
 'pieniędzy': 12253,
 'który': 12139,
 'masz': 11701,
 'będzie': 11694,
 'mogą': 11671,
 'firmy': 10711,
 'również': 10526,
 'akcji': 10396,
 'bardzo': 10177,
 'ci': 998

### Apply a distortion function to the queries part of the corpus

In [ ]:
queries = load_dataset("clarin-knext/fiqa-pl", "queries")['queries']
queries = queries.map(lambda x: {'text': x['text'].lower()})
queries[:5]['text']

['co jest uważane za wydatek służbowy w podróży służbowej?',
 'wydatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
 'rozpoczęcie nowego biznesu online',
 '„dzień roboczy” i „termin płatności” rachunków',
 'nowy właściciel firmy – jak działają podatki dla firmy i osoby fizycznej?']

In [ ]:
import random
import string
def distort_word(word):

    operation = random.choice(['remove', 'add', 'change'])
    
    if operation == 'remove' and len(word) > 2:
        # Remove a random letter, ensuring it's an alphabetic character
        pos = random.randint(0, len(word) - 1)
        while not word[pos].isalpha():
            pos = random.randint(0, len(word) - 1)
        return word[:pos] + word[pos+1:]
    
    elif operation == 'add':
        # Add a random letter at a valid position (not after punctuation)
        pos = random.randint(0, len(word))
        while word[pos - 1] in string.punctuation:
            pos = random.randint(0, len(word))
        new_letter = random.choice(string.ascii_lowercase)
        return word[:pos] + new_letter + word[pos:]
    
    elif operation == 'change':
        # Change a random letter to another random letter, ensuring it's an alphabetic character
        pos = random.randint(0, len(word) - 1)
        while not word[pos].isalpha():
            pos = random.randint(0, len(word) - 1)
        new_letter = random.choice(string.ascii_lowercase)
        return word[:pos] + new_letter + word[pos+1:]
    
    return word

def distort_query(query):
    words = query.split()
    if not words:
        return query
    
    # Filter out invalid words (single letters or words containing digits)
    valid_word_indices = [i for i, word in enumerate(words) if len(word) > 1 and word.isalpha()]
    
    if not valid_word_indices:
        return query
    
    word_changed = False
    while not word_changed:
        random_word_idx = random.choice(valid_word_indices)
        original_word = words[random_word_idx]
        new_word = distort_word(original_word)
        
        if new_word != original_word:  # Check if the word was actually changed
            words[random_word_idx] = new_word
            word_changed = True 
    return ' '.join(words)


In [9]:
queries = queries.map(lambda x: {'text': distort_query(x['text'])})

print(queries[:5]['text'])

['co jest uważane za wydatek służbowy w podróż służbowej?', 'wyoatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej', 'rozpoczęcie nowego biznesu oline', '„dzień roboczy” i „termin płatności” achunków', 'nowy błaściciel firmy – jak działają podatki dla firmy i osoby fizycznej?']


### Compute nDCG@10 for the distorted queris

In [10]:
qrels = load_dataset("clarin-knext/fiqa-pl-qrels")['test']

In [11]:
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch, helpers

load_dotenv(override=True)

username = os.getenv('ES_USERNAME')
password = os.getenv('ES_PASSWORD')

es = Elasticsearch(
    "http://localhost:9200",
    http_auth=(username, password)
)
es.info()['version']

C:\Users\katar\AppData\Local\Temp\ipykernel_12620\2515375146.py:10: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


{'number': '8.15.3',
 'build_flavor': 'default',
 'build_type': 'zip',
 'build_hash': 'f97532e680b555c3a05e73a74c28afb666923018',
 'build_date': '2024-10-09T22:08:00.328917561Z',
 'build_snapshot': False,
 'lucene_version': '9.11.1',
 'minimum_wire_compatibility_version': '7.17.0',
 'minimum_index_compatibility_version': '7.0.0'}

In [12]:
index_body = {
  "settings": {
    "analysis": {
      "analyzer": {
        "custom_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "morfologik_stem",
            "lowercase",
          ]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "text": {
        "type": "text",
        "analyzer": "custom_analyzer"  
      }
    }
  }
}

index_name = "qa_index_nodcg"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=index_body)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'qa_index_nodcg'})

In [13]:
actions = corpus.map(lambda item: {
    "_op_type": "index",
    "_index": "qa_index_nodcg",
    "_source": {
        "text": item["text"],
        "id": item["_id"]
    }
}).to_list()
helpers.bulk(es, actions)

(57638, [])

In [14]:
import math
import numpy as np
def execute_query(query, analyzer, field):
    response = es.search(
        index="qa_index_nodcg",
        body={
            "query": {
                "match": {
                    field: {
                        "query": query,
                        "analyzer": analyzer
                    }
                }
            },
            "size": 10
        }
    )
    return [hit["_id"] for hit in response["hits"]["hits"]]

def count_ndcg10(answers, correct_answers):
    DCG = sum(
        1 / math.log(i + 2, 2) for i in range(min(len(answers), 10)) if int(answers[i]) in correct_answers
    )
    IDCG = sum(
        1 / math.log(i + 2, 2) for i in range(min(len(correct_answers), 10))
    )
    return DCG / IDCG if IDCG > 0 else 0.0

def find_answers(query):
    answers = []
    answers.append(execute_query(query, "custom_analyzer", "text"))
    return answers

def calculate_results(answers, ids_correct):
    results = []
    for answer in answers:
        results.append(count_ndcg10(answer, ids_correct))
    return results


def perform_queries(queries, dataset_qa):
    results = []
    unique_query_ids = set(dataset_qa['query-id'])
    for query_id in unique_query_ids:
        question = queries.filter(lambda x: x['_id'] == str(query_id))['text'][0]
        answers = find_answers(question)
        
        correct_ids = qrels.filter(lambda x: x['query-id'] == query_id)['corpus-id']
        res = calculate_results(answers, correct_ids)
        results.append(res)
    return np.mean(results)

start = time.time()
baseline_result = perform_queries(queries, qrels)
baseline_time = (time.time() - start)
print(f"Mean NDCG@10 for baseline: {baseline_result}")
print(f"Execution time for baseline: {baseline_time}")

Mean NDCG@10 for baseline: 0.18945613485446425
Execution time for baseline: 7.9042346477508545


### Install Morfeusz and use it to find all words from the queries that do not appear in that dictionary.

In [15]:
import morfeusz2
morfeusz = morfeusz2.Morfeusz()
def find_incorrect_forms(queries):
    incorrect_forms = set()
    
    for query in queries:
        analysis = morfeusz.analyse(query['text'])
        for _, _, form in analysis:
            base_form, _, tag, _, _ = form
            if 'ign' in tag:  # Niepoprawna forma
                incorrect_forms.add(base_form)
    
    return incorrect_forms

In [16]:
incorrect_forms = find_incorrect_forms(queries)
incorrect_forms

{'dostosnwuje',
 'dtfinicja',
 'ktuórych',
 'ljak',
 'wymjść',
 'zmianz',
 'onetwotrade',
 'wpły',
 'ymiany',
 'ema',
 'twpłata',
 'ei',
 'cgzy',
 'yahood',
 'czx',
 'skrhcić',
 'uzvnanego',
 'dea',
 'financm',
 'spotify',
 'kontraktxmi',
 'przechowywiania',
 'kredyową',
 'sbwoim',
 'czs',
 'tendincję',
 'fnirmy',
 'sxwojego',
 'jaky',
 'okligacjami',
 'powierniczwy',
 'opgcji',
 'facbooka',
 'qdla',
 'podatqki',
 'oprpcz',
 'rozoczęcie',
 'flagstar',
 'obowiązująa',
 'pomóżq',
 'kaarta',
 'pelefony',
 'zemortgage',
 'ubezyieczeniu',
 'główte',
 'krlki',
 'zczym',
 'internetowychh',
 'wysokihej',
 'uberze',
 'delu',
 'transacje',
 'times',
 'cby',
 'taxact',
 'blub',
 'investing',
 'baynk',
 'ecn',
 'sją',
 'tree',
 'nż',
 'fwyjaśnienie',
 'ewu',
 'pozyczi',
 'zf',
 'ceu',
 'bryanii',
 'xnie',
 'fimy',
 'schedule',
 'kxedy',
 'akcjonariszy',
 'ogóe',
 'wstrmymanie',
 'fizyczsnej',
 'przee',
 'wyrażeniza',
 'otwierjąc',
 'neruchomości',
 'vile',
 'definicoa',
 'wdlaczego',
 'długtermino

### Use Levenshtein distance and the frequency list, to determine the most probable correction of the words in the queries that were identified as invalid. 

In [17]:
WORDS = global_frequency_list

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS.get(word, 0) / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'aąbcćdeęfghijklłmnńoópqrsśtuvwxyzźż'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [18]:
def transform_word_with_levenshtein(word):
    new_word = correction(word)
    return new_word

def transform_query_with_levenshtein(query):
    final_query = ""
    for token in nlp(query):
        if token.text not in incorrect_forms:
            final_query += token.text + " "
        else:
            corrected = transform_word_with_levenshtein(token.text)
            final_query += corrected + " "
    return final_query.strip()

In [ ]:
def correct_query(example):
    example['text'] = transform_query_with_levenshtein(example['text'])
    return example

start = time.time()
corrected_queries = queries.map(correct_query)
print(corrected_queries[:5]['text'])
levenshtein_correction_time = (time.time() - start)
print(f"Levenshtein correction time: {levenshtein_correction_time}")

Levenshtein correction time: 1.7020912170410156


### Compute nDCG@10 for your implementation of the spelling correction method.

In [20]:
start = time.time()
levenshtein_result = perform_queries(corrected_queries, qrels)
levenshtein_time = (time.time() - start + levenshtein_correction_time)
print(f"Mean NDCG@10 for Levenshtein: {levenshtein_result}")
print(f"Execution time for Levenshtein: {levenshtein_time}")

Mean NDCG@10 for Levenshtein: 0.2050057335620322
Execution time for Levenshtein: 8.845736742019653


### Use ElasticSearch's fuzzy match and compute nDCG@10 for this approach.

In [21]:
def execute_fuzzy_query(query, query_settings, field="text"):
    response = es.search(
        index="qa_index_nodcg",
        body={
            "query": {
                "match": {
                    field: {
                        "query": query,
                        "analyzer": "custom_analyzer",
                        **query_settings 
                    }
                }
            },
            "size": 10
        }
    )
    return [hit["_id"] for hit in response["hits"]["hits"]]


def find_fuzzy_answers(query, query_settings):
    answers = []
    answers.append(execute_fuzzy_query(query, query_settings))
    return answers

def perform_fuzzy_queries(queries, dataset_qa, query_settings):
    results = []
    unique_query_ids = set(dataset_qa['query-id'])
    for query_id in unique_query_ids:
        question = queries.filter(lambda x: x['_id'] == str(query_id))['text'][0]
        answers = find_fuzzy_answers(question, query_settings)
        
        correct_ids = qrels.filter(lambda x: x['query-id'] == query_id)['corpus-id']
        res = calculate_results(answers, correct_ids)
        results.append(res)
    return np.mean(results)

In [22]:
print(f"Mean NDCG@10 for baseline: {baseline_result}")
print(f"Execution time for baseline: {baseline_time}")
print(f"Mean NDCG@10 for Levenshtein: {levenshtein_result}")
print(f"Execution time for Levenshtein: {levenshtein_time}")
query_settings = [{"fuzziness": 'AUTO', "max_expansions": 5}, 
                  {"fuzziness": 2, "max_expansions": 2}, 
                  {"fuzziness": 2, "max_expansions": 2}, 
                  {"fuzziness": 2, "max_expansions": 1}]
for settings in query_settings:
    start = time.time()
    fuzzy_results = perform_fuzzy_queries(queries, qrels, settings)
    fuzzy_time = (time.time() - start)
    print(f"Mean NDCG@10 for fuzziness {settings['fuzziness']}, max expansion {settings['max_expansions']}: {fuzzy_results}")
    print(f"Execution time for fuzziness {settings['fuzziness']}, max expansion {settings['max_expansions']}: {fuzzy_time}")

Mean NDCG@10 for baseline: 0.18945613485446425
Execution time for baseline: 7.9042346477508545
Mean NDCG@10 for Levenshtein: 0.2050057335620322
Execution time for Levenshtein: 8.845736742019653


Filter: 100%|██████████| 1706/1706 [00:00<00:00, 93992.78 examples/s]


Mean NDCG@10 for fuzziness AUTO, max expansion 5: 0.18911328842630562
Execution time for fuzziness AUTO, max expansion 5: 83.89259576797485
Mean NDCG@10 for fuzziness 2, max expansion 2: 0.19548356488449933
Execution time for fuzziness 2, max expansion 2: 25.639511585235596
Mean NDCG@10 for fuzziness 2, max expansion 2: 0.19548356488449933
Execution time for fuzziness 2, max expansion 2: 24.97104024887085
Mean NDCG@10 for fuzziness 2, max expansion 1: 0.20064024914877182
Execution time for fuzziness 2, max expansion 1: 18.395487785339355


### Compare the results of baseline with the 2 implemented methods. Take into account the nDCG score and the performance of the methods

Jak widać z przeprowadzonych eksperymentów najlepsze wyniki, zarówno pod względem wartości nDCG, jak i czasu wykonania zostały uzyskane za pomocą automatu Levenshteina. Wartość nDCG@10 w tym przypadku wyniosła 20,5%, a czas wynonania (biorąc pod uwagę zarówno przekształcenia wyrazów, jak i obliczenie metryki) niecałe 9s. Zastosowanie fuzzy match w ElasticSearch z odpowiednimi parametrami (fuzziness=2, max_espansion=1) pozwoliło na uzyskanie zbliżonej wartości nDCG@10 (20%), jednak w dwukrotnie dłuższym czasie (18s). W przypadku metody podstawowej wartość nDCG@10 wyniosła niecałe 19%, a czas wykonania 8s. Widać zatem, że zastosowane metody nie zdołały poprawić tego wyniku w sposób znaczący, co może wynikać ze specyfiki danych.

### Use an LLM of your choice (you can use Bielik) to fix 30 first queries from the distorted set and compare the results manually with the method based on the Levenshtein distance.

In [26]:
original_queries = load_dataset("clarin-knext/fiqa-pl", "queries")['queries']
original_queries[:30]['text']

['Co jest uważane za wydatek służbowy w podróży służbowej?',
 'Wydatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
 'Rozpoczęcie nowego biznesu online',
 '„Dzień roboczy” i „termin płatności” rachunków',
 'Nowy właściciel firmy – Jak działają podatki dla firmy i osoby fizycznej?',
 'Hobby kontra biznes',
 'Czeki osobiste zamiast firmowych',
 'Czy amerykański kodeks podatkowy wymaga, aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty?',
 'Jak mogę zarejestrować firmę w Wielkiej Brytanii bez podawania adresu firmy?',
 'Czym są „podstawy biznesowe”?',
 'Strata na inwestycjach biznesowych z poprzedniego roku',
 'Jak mogę oszacować podatki biznesowe / opłaty za zgłoszenie dla firmy, która ma 0 USD dochodu?',
 'Czy zakup samochodu dla firmy za pomocą kredytu biznesowego zostałby uznany za wydatek biznesowy?',
 'Odliczanie strat biznesowych z ostatnich lat (nieudokumentowanych)',
 '30% udziału w bi

In [25]:
distorted_queries_30 = queries[:30]['text']
distorted_queries_30

['co jest uważane za wydatek służbowy w podróż służbowej?',
 'wyoatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
 'rozpoczęcie nowego biznesu oline',
 '„dzień roboczy” i „termin płatności” achunków',
 'nowy błaściciel firmy – jak działają podatki dla firmy i osoby fizycznej?',
 'phobby kontra biznes',
 'czeki osobiste zamiat firmowych',
 'czy amerykański kodeks podatkowy wymaga, aby właściciele małych fqirm liczyli zakupy biznesowe jako dochód osobisty?',
 'jak mogę zarejestrować firmę w wielkiej brytanii bez podawania adreu firmy?',
 'uzym są „podstawy biznesowe”?',
 'strata na inwestycjach ubiznesowych z poprzedniego roku',
 'jak mogę oszacować podatki biznesowe / opłaty za zgłoszenie dla firmy, któran ma 0 usd dochodu?',
 'czy zakup samochodu dla efirmy za pomocą kredytu biznesowego zostałby uznany za wydatek biznesowy?',
 'odliczanie strat biznesowych z osttnich lat (nieudokumentowanych)',
 '30% udzivłu w biz

In [24]:
levenshtein_corrected_queries = corrected_queries[:30]['text']
levenshtein_corrected_queries

['co jest uważane za wydatek służbowy w podróż służbowej ?',
 'wydatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek , który wydarzył się podczas podróży służbowej',
 'rozpoczęcie nowego biznesu online',
 '„ dzień roboczy ” i „ termin płatności ” rachunków',
 'nowy właściciel firmy – jak działają podatki dla firmy i osoby fizycznej ?',
 'hobby kontra biznes',
 'czeki osobiste zamiast firmowych',
 'czy amerykański kodeks podatkowy wymaga , aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty ?',
 'jak mogę zarejestrować firmę w wielkiej brytanii bez podawania adres firmy ?',
 'czym są „ podstawy biznesowe ” ?',
 'strata na inwestycjach biznesowych z poprzedniego roku',
 'jak mogę oszacować podatki biznesowe / opłaty za zgłoszenie dla firmy , która ma 0 usd dochodu ?',
 'czy zakup samochodu dla firmy za pomocą kredytu biznesowego zostałby uznany za wydatek biznesowy ?',
 'odliczanie strat biznesowych z ostatnich lat ( nieudokumentowanych )',
 '

**Prompt**:

Poniżej znajduje się lista 30 zapytań. W każdym zapytaniu jedno ze słów zawiera błąd (dodanie litery/usunięcie litery/zamiana litery na inną). Zidentyfikuj błędne słowa, popraw je i zwróć poprawione zapytania.

**Odpowiedź Bielika**:

Oto poprawione wersje zapytań z zidentyfikowanymi błędami:

    Co jest uważane za wydatek służbowy w podróży służbowej?

    Wyjątki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej.

    Rozpoczęcie nowego biznesu online.

    „Dzień roboczy” i „termin płatności” faktur.

    Nowy właściciel firmy - jak działają podatki dla firmy i osoby fizycznej?

    Hobby kontra biznes.

    Czeki osobiste zamiast firmowych.

    Czy amerykański kodeks podatkowy wymaga, aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty?

    Jak mogę zarejestrować firmę w Wielkiej Brytanii bez podawania adresu firmy?

    Czy istnieją „podstawy biznesowe”?

    Strata na inwestycjach biznesowych z poprzedniego roku.

    Jak mogę oszacować podatki biznesowe / opłaty za zgłoszenie dla firmy, która ma 0 USD dochodu?

    Czy zakup samochodu dla firmy za pomocą kredytu biznesowego zostałby uznany za wydatek biznesowy?

    Odliczanie strat biznesowych z ostatnich lat (nieudokumentowanych).

    30% udziału w biznesie.

    Odbieranie wydatków służbowych z osobistej karty kredytowej.

    Używanie CV-eku biznesowego do płatności w handlu detalicznym.

    Podatek przy zakładaniu firmy w pełnym wymiarze czasu pracy.

    Czy mogę spłacić saldo karty kredytowej, aby zwolnić dostępny kredyt?

    Wolne rozpoczynanie działalności pobocznej.

    Dlaczego „spieniężanie czeków” jest legalną działalnością?

    Czy biznes to jedyny sposób na zostanie milionerem?

    Wycenianie małej firmy, w którą warto zainwestować.

    Rozpoczęcie dużego biznesu z niezbyt dużymi dochodami?

    Wymagania dotyczące rozliczania podatków biznesowych?

    Rada do pracy i mojej firmy.

    Doradztwo w zakresie transferu pieniędzy.

    Jak przenieść leasing samochodu osobistego do leasingu samochodu biznesowego?

    Korepetycje z zarządzania płacami biznesowymi.

    Czy jako właściciel małej firmy powinienem płacić podatki z mojego osobistego lub firmowego konta czekowego?


W niektórych przypadkach Bielik poradził sobie lepiej niż automat Leveshteina, co jest dość spodziewanym efektem, ponieważ modele językowe mają możliwość analizowania kontekstu całego zapytania, nie tylko pojedynczego słowa, jak w prztpadku automatu Levenshteina. Z tego względu np. w zdaniu *cz jako właściciel małej firmy powinienem płacić podatki z mojego osobistego lub firmowego konta czekowego?* Bielik poprawnie zidentyfikował, że pierwszym słowem powinno byc *czy*, co nie udało się w przypaku automaty Levenshteina. Bielik lepiej poradził sobie również w przypadku słów, które istnieją w słowniku, jednak w wyniku przekształceń ich forma w zdaniu była niepoprawna, jak np. w zdaniu *co jest uważane za wydatek służbowy w podróż służbowej?* Bielik poprawnie zamienił formę *podróż* na *podróży*, natomiast w przypadku automatu Levenshteina forma ta pozostała niezmieniona. Z drugiej strony w niektórych przypadkach Bielik wykazywał się nadmierną kreatywnością i nie proponował najbliższych słów, np. zamiana *achunków* na *faktur* zamiast *rachunków* lub *wyoatki* na *wyjątki* zamiast *wydatki*. 

### Draw conclusions regarding:

1. the distribution of words in the corpus,
2. the performance of your method compared to ElasticSearch,
3. the results provided by your method compared to ElasticSearch,
4. the validity of the obtained corrections,
5. ability of an LLM to fix invalid queries.


1. Rozkład słów w korpusie jest zgodny z typowym rozkładem słów w języku: najczęściej występują słowa o niewielkiej wartości semantycznej, takie jak przyimki, spójniki czy zaimki. Natomiast rzadziej pojawiające się wyrazy są zazwyczaj bardziej specyficzne i niosą większe znaczenie. 
2. Z przeprowadzonych eksperymentów wynika, że metoda oparta na automacie Levenshteina przewyższa ElasticSearch pod względem szybkości działania. Levenshtein osiągnął nDCG@10 na poziomie 20,5% w czasie 9 sekund, podczas gdy ElasticSearch (z fuzziness=2 i max_expansion=1) uzyskał nDCG@10 wynoszące 20%, ale w czasie 18 sekund.
3. Metoda oparta na automacie Levenshteina zdołała osiągnąć lepszy wynik nDCG@10 (20,5%) w prównaniu z ElasticSearchem (20%). Dla parametrów fuzziness=2 i max_expansion=1 różnica ta jest niewielka, ale dla eksperymentów z innymi wartościami parametrów ElasticSearch radził sobie jeszcze gorzej i wymagał zdecydowanie więcej czasu.
4. Korekcje zapytań dokonane przez automat Levenshteina w większości przypadków były poprawne, jednak problemy pojawiały się w przypadku krótkich słów, z których usunięto literę, np. czy -> cz nie zostało poprawnie zrekonstruowane. Problematyczne były też skróty i skrótowce, w których wprowadzono zmiany czy wyrazy, które występują w słowniku, jednak w wynikiu usunięcia/dodania litery ich forma gramatyczna w kontekście danego zapytania stała się niepoprawna. 
5. Modele językowe dobrze radzą sobie z poprawianiem literówek ze względu na to, że mają możliwość analizowania kontekstu całego zdania, nie tylko pojedynczego wyrazu, co daje im przewagę w porównaniu z rozwiązaniami taki jak automat Leveshteina.